In [1]:
%pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 7.0 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149863 sha256=0632cc0523ee68cceb992c952cba44b2a1379d00c33153745cc2363017cfb80b
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [2]:
import pickle
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Reshape, Dense, Bidirectional, Dropout, LSTM 
,StringLookup, ConvLSTM2D)
from tensorflow.keras import Model
from tensorflow.keras.backend import ctc_decode, ctc_batch_cost
from tensorflow.keras.layers import Layer

import Levenshtein as lev

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
with open('./labeltensor_shpad.pkl','rb') as ry:
    lab_tensor_sh = pickle.load(ry)

with open('./drive/MyDrive/imgtensor_trsh.pkl','rb') as arbi:
    img_tensor_trsh = pickle.load(arbi)

charlist = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,-+1234567890'

def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    #trying this method here-  
    #https://stackoverflow.com/questions/64321779/how-to-use-tf-ctc-loss-with-variable-length-features-and-labels
    label_length = tf.math.count_nonzero(y_true, axis= -1, keepdims = True)
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return tf.reduce_mean(loss)

def decode_preds(pred_model,start,end):
    #preds = pred_model.predict(img_tensor_trsh[start:end])
    preds = pred_model
    len_tensor = tf.convert_to_tensor([len([l for l in lab if l>0]) for lab in lab_tensor_sh[start:end]])
    pred_texts = ctc_decode(preds,len_tensor,greedy=True)
    guess_list =  [''.join([charlist[p-1] for p in pred.numpy() if p > -1]) for pred in pred_texts[0][0]]
    prob_list = pred_texts[1].numpy()
    return guess_list,prob_list

def post_distance(pred_model,start,end):
    preds, _ = decode_preds(pred_model,start,end)
    trues = [''.join([charlist[p-1] for p in lab.numpy() if p>0]) for lab in lab_tensor_sh[start:end]]
    return np.mean([lev.distance(trues[i],preds[i]) for i in range(end-start)])
# you can normalize the lev distance by dividing by length of longest word, but we'll leave that for now

In [5]:
input_img = Input(shape=(679,480,1),name='image',dtype="float32")


x = Conv2D(32,(3,3),activation='relu',name='Conv1')(input_img)
x = MaxPooling2D((2,2), name = "pool1")(x)
x = Dropout(0.3,name = 'drop1')(x)


x = Conv2D(64,(3,3),activation='relu',name='Conv2')(x)
x = MaxPooling2D((2,2), name = "pool2")(x)
x = Dropout(0.3, name = 'drop2')(x)

x = Conv2D(128,(4,4), activation= 'relu',name='Conv3')(x)
x = Conv2D(128,(5,5),activation='relu', name = 'Conv4')(x)
x = MaxPooling2D((2,2), name = 'pool3')(x)
x = Dropout(0.3,name = 'drop3')(x)

x = Conv2D(128,(5,5), activation= 'relu', name = 'Conv5')(x)

shape = (152,3264)

x = Reshape(target_shape=shape, name = 'reshape')(x)
x = Dense(64, activation='relu', name ='dense1')(x)
x = Dropout(0.2,name = 'drop4')(x)

x = Bidirectional(LSTM(256,return_sequences=True, dropout=0.25), name = 'LSTM1')(x)
x = Bidirectional(LSTM(128,return_sequences=True,dropout=0.25), name = 'LSTM2')(x)
x = Bidirectional(LSTM(64,return_sequences=True,dropout=0.25), name = 'LSTM3')(x)
x = Bidirectional(LSTM(32,return_sequences=True,dropout=0.25), name = 'LSTM4')(x)


output = Dense(len(charlist)+1,activation='softmax',name='dense2')(x)


model = Model(input_img,output,name='basic-OCRplusvartr')
model.compile(optimizer='adam',loss=CTCLoss) #try different optimizers, lstm rows, etc

In [6]:
eps = 100
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)
history = model.fit(img_tensor_trsh,lab_tensor_sh,validation_split=0.2, epochs=eps,callbacks=[early_stopping])

Epoch 1/100
36/36 [==============================] - 60s 782ms/step - loss: 109.5611 - val_loss: 41.5365
Epoch 2/100
36/36 [==============================] - 22s 620ms/step - loss: 34.2780 - val_loss: 35.7414
Epoch 3/100
36/36 [==============================] - 22s 626ms/step - loss: 32.4258 - val_loss: 35.3739
Epoch 4/100
36/36 [==============================] - 23s 631ms/step - loss: 32.1442 - val_loss: 35.1295
Epoch 5/100
36/36 [==============================] - 23s 637ms/step - loss: 32.0043 - val_loss: 35.0314
Epoch 6/100
36/36 [==============================] - 23s 635ms/step - loss: 31.8564 - val_loss: 35.0595
Epoch 7/100
36/36 [==============================] - 23s 634ms/step - loss: 31.8492 - val_loss: 34.8216
Epoch 8/100
36/36 [==============================] - 23s 634ms/step - loss: 31.6902 - val_loss: 34.5942
Epoch 9/100
36/36 [==============================] - 23s 637ms/step - loss: 31.4027 - val_loss: 34.5701
Epoch 10/100
36/36 [==============================] - 23s 634ms

In [7]:
preds = Model(model.get_layer(name="image").input, model.get_layer(name='dense2').output)

In [8]:
guesses, probs = decode_preds(preds,0,1200)

In [9]:
post_distance(preds,0,1407)

7.334044065387349

In [10]:
trues = [''.join([charlist[p-1] for p in  lab.numpy() if p>0]) for lab in lab_tensor_sh[:1200]]

In [12]:
trues[:20], guesses[:20]

(['Secretary',
  'Neg',
  '--- Steven A',
  '36',
  'M',
  'Flowers Ruby J',
  '---, Frank H',
  '---, Vera P',
  'Edwards  Ada',
  'Welch Ernestine',
  'Davis Henry A',
  'Alabama',
  'W',
  'Head',
  'Steward Mildred R',
  '69',
  'M',
  'H',
  'No one at home',
  'Greece'],
 ['N',
  '2',
  '--',
  '2',
  'F',
  'N',
  '--',
  '--',
  'C',
  'C',
  'C',
  'Al',
  'W',
  'H',
  'C',
  '2',
  'F',
  'F',
  'No',
  'L'])

In [13]:
hist2 = model.fit(img_tensor_trsh,lab_tensor_sh,validation_split=0.2, epochs=eps,callbacks=[early_stopping])

Epoch 1/100
36/36 [==============================] - 23s 630ms/step - loss: 17.7521 - val_loss: 22.6847
Epoch 2/100
36/36 [==============================] - 23s 631ms/step - loss: 17.5952 - val_loss: 22.7641
Epoch 3/100
36/36 [==============================] - 23s 627ms/step - loss: 17.7223 - val_loss: 22.7449
Epoch 4/100
36/36 [==============================] - 22s 625ms/step - loss: 17.7332 - val_loss: 22.8386
Epoch 5/100
36/36 [==============================] - 22s 626ms/step - loss: 17.6382 - val_loss: 23.1981
Epoch 6/100
36/36 [==============================] - 23s 628ms/step - loss: 17.5084 - val_loss: 22.9932
Epoch 7/100
36/36 [==============================] - 23s 627ms/step - loss: 17.3444 - val_loss: 22.8106
Epoch 8/100
36/36 [==============================] - 23s 627ms/step - loss: 17.7663 - val_loss: 23.3984
Epoch 9/100
36/36 [==============================] - 23s 627ms/step - loss: 17.5167 - val_loss: 22.8947
Epoch 10/100
36/36 [==============================] - 22s 626ms/

In [14]:
preds = Model(model.get_layer(name="image").input, model.get_layer(name='dense2').output)

In [15]:
guesses, probs = decode_preds(preds,0,1200)

In [21]:
len([guesses[i] for i in range(len(guesses)) if guesses[i] == trues[i]])

95

In [27]:
np.mean([len(guess) for guess in guesses])

1.2683333333333333

In [29]:
!unzip fm2.zip -d ./firstmodel2/

Archive:  fm2.zip
   creating: ./firstmodel2/first_model_200/
  inflating: ./firstmodel2/first_model_200/saved_model.pb  
  inflating: ./firstmodel2/first_model_200/keras_metadata.pb  
   creating: ./firstmodel2/first_model_200/assets/
   creating: ./firstmodel2/first_model_200/variables/
  inflating: ./firstmodel2/first_model_200/variables/variables.data-00000-of-00001  
  inflating: ./firstmodel2/first_model_200/variables/variables.index  


In [32]:
model2 = tf.keras.models.load_model('./firstmodel2/first_model_200', custom_objects={'CTCLoss':CTCLoss})

In [35]:
with open('./drive/MyDrive/imgtensor_sh.pkl','rb') as arbi:
    img_tensor_sh = pickle.load(arbi)

In [36]:
pred_model = model2.predict(img_tensor_sh)

In [41]:
guessog, predog = decode_preds(pred_model,0,1407)

In [43]:
np.mean([lev.distance(guessog[i],trues[i]) for i in range(len(trues))])

5.0625

In [44]:
max([lev.distance(guessog[i],trues[i]) for i in range(len(trues))])

33

In [45]:
len([lev.distance(guessog[i],trues[i]) for i in range(len(trues)) if lev.distance(guessog[i],trues[i])==0])

261

In [46]:
offby = [lev.distance(guessog[i],trues[i]) for i in range(len(trues))]

In [47]:
from collections import Counter

In [48]:
Counter(offby)

Counter({0: 261,
         1: 205,
         2: 127,
         3: 13,
         4: 44,
         5: 62,
         6: 64,
         7: 78,
         8: 46,
         9: 37,
         10: 56,
         11: 38,
         12: 40,
         13: 35,
         14: 29,
         15: 22,
         16: 16,
         17: 6,
         18: 6,
         19: 2,
         20: 2,
         21: 3,
         22: 3,
         23: 1,
         25: 1,
         26: 1,
         31: 1,
         33: 1})

In [50]:
trulen = [len(true) for true in trues]

In [51]:
Counter(trulen)

Counter({1: 158,
         2: 142,
         3: 87,
         4: 98,
         5: 13,
         6: 32,
         7: 89,
         8: 65,
         9: 41,
         10: 48,
         11: 66,
         12: 59,
         13: 58,
         14: 72,
         15: 37,
         16: 33,
         17: 36,
         18: 28,
         19: 11,
         20: 5,
         21: 6,
         22: 2,
         23: 2,
         24: 1,
         25: 3,
         26: 1,
         27: 1,
         28: 2,
         30: 1,
         31: 1,
         36: 1,
         37: 1})